In [1]:
from pandas_datareader import data as pdr
import FinanceDataReader as fdr
import yfinance as yf
from pykrx import stock
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

# 함수

In [2]:
# 강사님 차트 함수
def myplot(df, nrows=5,  kind='line', title='제목', labels=False):  
#     if nrows > 0:
#         print(df.head(nrows))
#     else:
#         print(df.tail(nrows))
    if labels:
        cols = df.columns
        for i, col in enumerate(cols):
            df[col].plot(label=labels[i], kind=kind)
    else :
        df.plot(kind=kind)
    
    plt.title(title)
    plt.legend()
    plt.show()


In [3]:
# 강사님 차트 함수
def myplotmix(df1, df2, y1='ylabel-1', y2='ylabel-2', kind1='bar', kind2='line', title='제목',  nrows1=5, nrows2=5,  labels=False):  
    ytl=['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

    if nrows1 > 0:
        print(df1.head(nrows1))
    elif nrows1 < 0:
        print(df1.tail(nrows1))
    if nrows2 > 0:
        print(df2.head(nrows2))
    else:
        print(df2.tail(nrows2))
    fig, ax1 = plt.subplots(figsize=(12, 4))
    ax2=ax1.twinx()
    if kind1 == 'line':
        df1.plot(kind=kind1, ax=ax2, marker='d',color='#e35f62')  #color='y', 
        df2.plot(kind=kind2, ax=ax1)  #bar
    else:
        df1.plot(kind=kind1, ax=ax1,color='#bcbd22')
        df2.plot(kind=kind2, ax=ax2) #color='y',
        
    # df['date'] = df['date'].dt.strftime('%Y-%m-%d') 
    # ax1.set_xticklabels(df['date'])    
    ax1.yaxis.tick_right()
    ax2.yaxis.tick_left()
    ax1.set_ylabel(y1) #, color='red')
    ax2.set_ylabel(y2) #, color='red')
    ax1.legend(loc=1)
    ax2.legend(loc=2)
    plt.title(title)
    ax1.grid(False)
    
    plt.show()

- 데이터 불러오기

In [4]:
df = pd.read_excel('WTI선물.xlsx', sheet_name='M')
expdf = pd.read_excel('WTI선물.xlsx', sheet_name='EXP')

In [24]:
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2014 non-null   datetime64[ns]
 1   Spot    2014 non-null   float64       
 2   Front   2014 non-null   float64       
 3   Second  2014 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 63.1 KB


# 이론가 산출
이론가 = 지수가격 * EXP( (cd금리) * t)

In [6]:
def 이론가(선물지수, 이자율, 배당률, 오늘, 만기일):
    선물이론가 = 선물지수 * np.exp((이자율-배당률) * np.busday_count(오늘, 만기일)/252)
    return 선물이론가

In [7]:
# sample data
temp = pd.DataFrame({
                    'date':pd.date_range('2021-01-04',periods=4),
                   '지수': [100,101,102,103]   
#                    'b':[1,1,1,1]
                })

temp['date']  = pd.to_datetime(temp['date'] , format='%Y-%m-%d', errors='raise')
temp = temp.set_index('date')

In [8]:
이자율 = 0.0225
배당률 = 0.015

for date in temp.index:
    오늘 = date.date()
    만기일 = pd.to_datetime('2021-01-08').date()
    선물지수 = temp.loc[date,'지수']
    선물이론가 = 이론가(선물지수,이자율,배당률,오늘, 만기일)
    temp.loc[date,'선물이론가']= 선물이론가
temp

,지수,선물이론가
date,,
2021-01-04,100,100.011905
2021-01-05,101,101.009018
2021-01-06,102,102.006072
2021-01-07,103,103.003066
